In [1]:
import os

In [2]:
%pwd

'c:\\Users\\rahul\\Downloads\\Facial_review_system\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\rahul\\Downloads\\Facial_review_system'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_classes: int
    
@dataclass(frozen=True)
class PrepareWeightsConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path

In [6]:
from Facial_Review_System.constants import *
from Facial_Review_System.utils.common import read_yaml, create_directories

In [7]:
#this class for base model configuration

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_weights_config(self) -> PrepareWeightsConfig:
        config = self.config.prepare_weights

        create_directories([config.root_dir])
        
        prepare_weights_config = PrepareWeightsConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file
        )


        return prepare_weights_config

In [8]:
import os
import zipfile
import gdown
from Facial_Review_System import logger
from Facial_Review_System.utils.common import get_size

In [9]:
# #this class is for model download
# class ModelDownload:
#     def __init__(self, config: PrepareBaseModelConfig):
#         self.config = config

    
#     def download_file(self)-> str:
#         '''
#         Fetch data from the url
#         '''

#         try: 
#             dataset_url = self.config.source_URL
#             os.makedirs("artifacts/prepare_base_model", exist_ok=True)
#             logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

#             file_id = dataset_url.split("/")[-2]
#             prefix = 'https://drive.google.com/uc?/export=download&id='
#             gdown.download(prefix+file_id,zip_download_dir)

#             logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

#         except Exception as e:
#             raise e

            
# #this class if for weights download

# class WeightsDownload:
#     def __init__(self, config: PrepareWeightsConfig):
#         self.config = config

    
#     def download_file(self)-> str:
#         '''
#         Fetch data from the url
#         '''

#         try: 
#             dataset_url = self.config.source_URL
#             os.makedirs("artifacts/prepare_base_model", exist_ok=True)
#             logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

#             file_id = dataset_url.split("/")[-2]
#             prefix = 'https://drive.google.com/uc?/export=download&id='
#             gdown.download(prefix+file_id,zip_download_dir)

#             logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

#         except Exception as e:
#             raise e


In [10]:
#by chatgpt
class ModelDownload:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def download_file(self) -> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            os.makedirs(self.config.root_dir, exist_ok=True)  # Updated directory path
            logger.info(f"Downloading data from {dataset_url} into file {self.config.local_data_file}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix + file_id, self.config.local_data_file)  # Save to local_data_file

            logger.info(f"Downloaded data from {dataset_url} into file {self.config.local_data_file}")

        except Exception as e:
            raise e


            
#this class if for weights download

class WeightsDownload:
    def __init__(self, config: PrepareWeightsConfig):
        self.config = config

    def download_file(self) -> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            os.makedirs(self.config.root_dir, exist_ok=True)  # Updated directory path
            logger.info(f"Downloading data from {dataset_url} into file {self.config.local_data_file}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix + file_id, self.config.local_data_file)  # Save to local_data_file

            logger.info(f"Downloaded data from {dataset_url} into file {self.config.local_data_file}")

        except Exception as e:
            raise e


In [11]:
try:
    config = ConfigurationManager()
    model_config = config.get_model_config() 
    vgg_model = ModelDownload(config = model_config)
    zip_download_dir = model_config.local_data_file 
    vgg_model.download_file()
    weight_config = config.get_prepare_weights_config()
    comp_weights = WeightsDownload(config=weight_config)
    zip_download_dir = weight_config.local_data_file 
    comp_weights.download_file()
except Exception as e:
    raise e

[2024-04-26 10:49:07,314: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-26 10:49:07,316: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-26 10:49:07,317: INFO: common: created directory at: artifacts]
[2024-04-26 10:49:07,318: INFO: common: created directory at: artifacts/prepare_base_model]
[2024-04-26 10:49:07,320: INFO: 411889403: Downloading data from https://drive.google.com/file/d/10oVyfXZwo0o_QsYPfYyFOHF6145IDDvn/view?usp=drive_link into file artifacts/prepare_base_model/base_model.h5]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=10oVyfXZwo0o_QsYPfYyFOHF6145IDDvn
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=10oVyfXZwo0o_QsYPfYyFOHF6145IDDvn&confirm=t&uuid=a918251b-eecf-4cb2-adea-a380af77afb3
To: c:\Users\rahul\Downloads\Facial_review_system\artifacts\prepare_base_model\base_model.h5
100%|██████████| 580M/580M [01:46<00:00, 5.43MB/s] 

[2024-04-26 10:50:56,053: INFO: 411889403: Downloaded data from https://drive.google.com/file/d/10oVyfXZwo0o_QsYPfYyFOHF6145IDDvn/view?usp=drive_link into file artifacts/prepare_base_model/base_model.h5]
[2024-04-26 10:50:56,054: INFO: common: created directory at: artifacts/prepare_weights]
[2024-04-26 10:50:56,055: INFO: 411889403: Downloading data from https://drive.google.com/file/d/12xz9oIG_wWH5B5hilQO5IXvHfs1hrVPJ/view?usp=sharing into file artifacts/prepare_weights/weights.h5]



Downloading...
From (original): https://drive.google.com/uc?/export=download&id=12xz9oIG_wWH5B5hilQO5IXvHfs1hrVPJ
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=12xz9oIG_wWH5B5hilQO5IXvHfs1hrVPJ&confirm=t&uuid=814dcc23-208c-429e-a2e6-f984801ec09d
To: c:\Users\rahul\Downloads\Facial_review_system\artifacts\prepare_weights\weights.h5
100%|██████████| 598M/598M [01:57<00:00, 5.09MB/s] 

[2024-04-26 10:52:55,418: INFO: 411889403: Downloaded data from https://drive.google.com/file/d/12xz9oIG_wWH5B5hilQO5IXvHfs1hrVPJ/view?usp=sharing into file artifacts/prepare_weights/weights.h5]
